# CodeAct: Code-Centric Agent Execution

In [1]:
import re
import io
from contextlib import redirect_stdout, redirect_stderr
from agentic_patterns.core.agents import get_agent, run_agent

## The CodeAct Pattern

CodeAct treats code execution as the agent's primary action modality. Instead of text-based actions like ReAct, the agent generates actual executable code. The code is run in a sandboxed environment, and the output (including errors) becomes feedback for the next iteration.

## Execution Sandbox

A simple sandbox that executes Python code and captures output. The namespace persists across executions, allowing the agent to build state incrementally.

In [2]:
class CodeSandbox:
    """Simple Python execution sandbox with persistent namespace."""

    def __init__(self):
        self.namespace = {"__builtins__": __builtins__}

    def execute(self, code: str, timeout: float = 5.0) -> str:
        """Execute code and return combined stdout/stderr output."""
        stdout_capture = io.StringIO()
        stderr_capture = io.StringIO()

        try:
            with redirect_stdout(stdout_capture), redirect_stderr(stderr_capture):
                exec(code, self.namespace)

            output = stdout_capture.getvalue()
            return output if output else "(code executed successfully, no output)"

        except Exception as e:
            return f"Error: {type(e).__name__}: {e}"

## CodeAct Prompt

The prompt instructs the model to think in code. Code blocks are executed, and results are fed back.

In [3]:
CODEACT_PROMPT = """You are a code-execution agent. Solve tasks by writing and executing Python code.

Rules:
1. Write code in ```python blocks. Each block will be executed and you'll see the output.
2. Use print() to show results - this is how you see what's happening.
3. Variables persist between executions, so you can build on previous code.
4. If you get an error, analyze it and fix your code in the next iteration.
5. When the task is complete, write DONE followed by a brief summary.

Example interaction:
User: Calculate the factorial of 5
Assistant: I'll calculate the factorial of 5.
```python
result = 1
for i in range(1, 6):
    result *= i
print(f"Factorial of 5 is {result}")
```
[Execution output: Factorial of 5 is 120]
DONE: The factorial of 5 is 120.
"""

## The CodeAct Loop

The loop extracts code blocks, executes them, and appends results to the context for the next iteration.

In [4]:
def extract_code_blocks(text: str) -> list[str]:
    """Extract Python code blocks from markdown-formatted text."""
    pattern = r"```python\s*(.*?)```"
    return re.findall(pattern, text, re.DOTALL)


async def codeact_loop(task: str, max_steps: int = 6) -> str:
    """Run the CodeAct loop until DONE or max steps reached."""
    agent = get_agent(system_prompt=CODEACT_PROMPT)
    sandbox = CodeSandbox()

    trajectory = f"Task: {task}\n"

    for step in range(max_steps):
        agent_run, _ = await run_agent(agent, trajectory)
        response = agent_run.result.output.strip()

        print(f"\n{'=' * 50}")
        print(f"Step {step + 1}")
        print(f"{'=' * 50}")
        print(response)

        trajectory += f"\nAssistant: {response}\n"

        # Extract and execute code blocks first
        code_blocks = extract_code_blocks(response)

        if code_blocks:
            for code in code_blocks:
                code = code.strip()
                if code:
                    print("\n--- Executing ---")
                    output = sandbox.execute(code)
                    print(f"Output: {output}")
                    trajectory += f"\n[Execution output: {output}]\n"

        # Check for completion after executing any code
        if "DONE" in response.upper():
            print(f"\n{'=' * 50}")
            print("Task completed")
            return response

        if not code_blocks:
            trajectory += "\n[No code block found. Please write Python code in ```python blocks.]\n"

    return "Max steps reached."

## Example 1: Data Analysis

The agent analyzes sales data by writing and executing code. Note how it builds state across executions.

In [5]:
task = """
Analyze this sales data and find:
1. Total revenue
2. Best selling product
3. Average order value

Data:
orders = [
    {"product": "Widget A", "quantity": 5, "price": 10.00},
    {"product": "Widget B", "quantity": 3, "price": 25.00},
    {"product": "Widget A", "quantity": 8, "price": 10.00},
    {"product": "Widget C", "quantity": 2, "price": 50.00},
    {"product": "Widget B", "quantity": 6, "price": 25.00},
]
"""

await codeact_loop(task)


Step 1
I'll analyze the sales data to find the total revenue, best selling product, and average order value.

```python
orders = [
    {"product": "Widget A", "quantity": 5, "price": 10.00},
    {"product": "Widget B", "quantity": 3, "price": 25.00},
    {"product": "Widget A", "quantity": 8, "price": 10.00},
    {"product": "Widget C", "quantity": 2, "price": 50.00},
    {"product": "Widget B", "quantity": 6, "price": 25.00},
]

# 1. Calculate total revenue
total_revenue = sum(order["quantity"] * order["price"] for order in orders)
print(f"1. Total Revenue: ${total_revenue:.2f}")

# 2. Find best selling product (by quantity sold)
product_quantities = {}
for order in orders:
    product = order["product"]
    quantity = order["quantity"]
    product_quantities[product] = product_quantities.get(product, 0) + quantity

best_selling_product = max(product_quantities, key=product_quantities.get)
best_selling_quantity = product_quantities[best_selling_product]
print(f"\n2. Best Selling Prod

'I\'ll analyze the sales data to find the total revenue, best selling product, and average order value.\n\n```python\norders = [\n    {"product": "Widget A", "quantity": 5, "price": 10.00},\n    {"product": "Widget B", "quantity": 3, "price": 25.00},\n    {"product": "Widget A", "quantity": 8, "price": 10.00},\n    {"product": "Widget C", "quantity": 2, "price": 50.00},\n    {"product": "Widget B", "quantity": 6, "price": 25.00},\n]\n\n# 1. Calculate total revenue\ntotal_revenue = sum(order["quantity"] * order["price"] for order in orders)\nprint(f"1. Total Revenue: ${total_revenue:.2f}")\n\n# 2. Find best selling product (by quantity sold)\nproduct_quantities = {}\nfor order in orders:\n    product = order["product"]\n    quantity = order["quantity"]\n    product_quantities[product] = product_quantities.get(product, 0) + quantity\n\nbest_selling_product = max(product_quantities, key=product_quantities.get)\nbest_selling_quantity = product_quantities[best_selling_product]\nprint(f"\\n2

## Example 2: Error Recovery

This example shows how the agent handles and recovers from errors, using them as feedback to fix the code.

In [6]:
task = """
Calculate the mean and standard deviation of these numbers: [4, 8, 15, 16, 23, 42]
Do not import any libraries - implement the calculations manually.
"""

await codeact_loop(task)


Step 1
I'll calculate the mean and standard deviation manually without importing any libraries.

```python
numbers = [4, 8, 15, 16, 23, 42]

# Calculate mean
n = len(numbers)
total = sum(numbers)
mean = total / n

print(f"Numbers: {numbers}")
print(f"Count: {n}")
print(f"Sum: {total}")
print(f"Mean: {mean}")
```

```python
# Calculate standard deviation
# Formula: sqrt(sum((x - mean)^2) / n)

# First, calculate sum of squared differences
squared_diffs = []
for x in numbers:
    diff = x - mean
    squared_diff = diff ** 2
    squared_diffs.append(squared_diff)
    print(f"({x} - {mean})^2 = {squared_diff}")

sum_squared_diffs = sum(squared_diffs)
print(f"\nSum of squared differences: {sum_squared_diffs}")
```

```python
# Calculate variance and standard deviation
variance = sum_squared_diffs / n
print(f"Variance: {variance}")

# Calculate square root manually using Newton's method
def sqrt(x, tolerance=1e-10):
    if x == 0:
        return 0
    guess = x / 2
    while True:
        b

'I\'ll calculate the mean and standard deviation manually without importing any libraries.\n\n```python\nnumbers = [4, 8, 15, 16, 23, 42]\n\n# Calculate mean\nn = len(numbers)\ntotal = sum(numbers)\nmean = total / n\n\nprint(f"Numbers: {numbers}")\nprint(f"Count: {n}")\nprint(f"Sum: {total}")\nprint(f"Mean: {mean}")\n```\n\n```python\n# Calculate standard deviation\n# Formula: sqrt(sum((x - mean)^2) / n)\n\n# First, calculate sum of squared differences\nsquared_diffs = []\nfor x in numbers:\n    diff = x - mean\n    squared_diff = diff ** 2\n    squared_diffs.append(squared_diff)\n    print(f"({x} - {mean})^2 = {squared_diff}")\n\nsum_squared_diffs = sum(squared_diffs)\nprint(f"\\nSum of squared differences: {sum_squared_diffs}")\n```\n\n```python\n# Calculate variance and standard deviation\nvariance = sum_squared_diffs / n\nprint(f"Variance: {variance}")\n\n# Calculate square root manually using Newton\'s method\ndef sqrt(x, tolerance=1e-10):\n    if x == 0:\n        return 0\n    gu